## Reproduce Figure 5b

In [2]:
import pandas as pd
import ipynbname
REPO_PATH = ipynbname.path().parent.parent

## Launch predictions using Variantformer

In [4]:
# Essential imports
import sys
from pathlib import Path
import warnings

warnings.filterwarnings("ignore")

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

from processors.variantprocessor import VariantProcessor

### Load data

In [6]:
variants = pd.read_parquet(os.path.join(REPO_PATH, '_artifacts/b23c9b69.pq'))
variants['tissue'].value_counts()

tissue
blood                              17724
skin                               14723
adipose                            14671
brain - frontal cortex (ba9)       13029
brain - putamen (basal ganglia)    12332
brain - substantia nigra           10604
Name: count, dtype: int64

### Predict for first 10 variants

In [7]:
# tiisue map to align tissue names between VF and Eqtl catalog data
vf_tissue_map = {'skin' : 'skin - sun exposed (lower leg)',
                 'blood' : 'whole blood',
                 'adipose' : 'adipose - subcutaneous',
                 'brain - frontal cortex (ba9)': 'brain - frontal cortex (ba9)',
                 'brain - putamen (basal ganglia)': 'brain - putamen (basal ganglia)',
                 'brain - substantia nigra' : 'brain - substantia nigra'
                 }
variants['tissue'] = variants['tissue'].map(vf_tissue_map)

In [8]:

first10_variants = variants.head(10)
first10_variants['chr'] = first10_variants['variant_id'].apply(lambda x: x.split('_')[0])
first10_variants['pos'] = first10_variants['variant_id'].apply(lambda x: int(x.split('_')[1]))
first10_variants['ref'] = first10_variants['variant_id'].apply(lambda x: x.split('_')[2])
first10_variants['alt'] = first10_variants['variant_id'].apply(lambda x: x.split('_')[3])


In [9]:
variants['tissue'].value_counts()

tissue
whole blood                        17724
skin - sun exposed (lower leg)     14723
adipose - subcutaneous             14671
brain - frontal cortex (ba9)       13029
brain - putamen (basal ganglia)    12332
brain - substantia nigra           10604
Name: count, dtype: int64

In [10]:
# Initialize VariantFormer
print("🚀 Initializing VariantFormer Variant Processor...")
model_class = 'v4_ag' # model class can be 'v4_ag', 'v4_pcg'. AG model is all-genes model trained on both protein-coding and non-coding genes.
# model_class = 'v4_pcg' # Uncomment to use the PCG model
vep = VariantProcessor(model_class=model_class)
# Run variant predictions
print("🔬 Running VariantFormer variant analysis...")
output_dir = "/tmp/vep_eqtl_output"
# Predict expression effects for all variants
raw_predictions = vep.predict(var_df=first10_variants, output_dir=output_dir)
print("Formatting VariantFormer scores and computing eQTL statistics...")
formatted_scores = vep.format_scores(raw_predictions)
print("Computing eQTL statistics...")
final_results = vep.eqtl_scores(formatted_scores)

2025-11-04 15:00:08 - processors.variantprocessor - INFO - Initializing Variant Processor...
2025-11-04 15:00:08 - processors.multi_datasets_loader - INFO - Loading gene annotations...
2025-11-04 15:00:08 - processors.multi_datasets_loader - INFO - Loading CRE annotations...


🚀 Initializing VariantFormer Variant Processor...
🔬 Running VariantFormer variant analysis...


2025-11-04 15:00:09 - processors.variantprocessor - INFO - Loading variants...
2025-11-04 15:00:09 - processors.multi_datasets_loader - INFO - Loaded 10 variants
2025-11-04 15:00:09 - processors.variantprocessor - INFO - Loaded 10 variants for processing
2025-11-04 15:00:09 - processors.variantprocessor - INFO - Mapped 10 gene-variant pairs
2025-11-04 15:00:09 - processors.variantprocessor - INFO - Loading BPE encoder...
2025-11-04 15:00:09 - processors.variantprocessor - INFO - Loading model...
2025-11-04 15:00:09 - processors.model_manager - INFO - Loading Seq2Reg model...


Loaded BPE vocabulary from /work/vocabs/bpe_vocabulary_500_using_huggingface.json


2025-11-04 15:00:10 - processors.model_manager - INFO - Loading Seq2Reg gene model...
2025-11-04 15:00:10 - processors.model_manager - INFO - Creating Seq2Gene model...
2025-11-04 15:00:15 - processors.model_manager - INFO - Model class: <class 'seq2gene.model_combined_modulator.Seq2GenePredictorCombinedModulator'>
2025-11-04 15:00:15 - processors.model_manager - INFO - Model architecture:
2025-11-04 15:00:15 - processors.model_manager - INFO - Model: Seq2GenePredictorCombinedModulator
2025-11-04 15:00:15 - processors.model_manager - INFO -   start_tkn: 96,768 params
2025-11-04 15:00:15 - processors.model_manager - INFO -   cre_tokenizer: 31,826,153 params
2025-11-04 15:00:15 - processors.model_manager - INFO -   gene_tokenizer: 31,826,153 params
2025-11-04 15:00:15 - processors.model_manager - INFO -   gene_map: 787,968 params
2025-11-04 15:00:15 - processors.model_manager - INFO -   cre_map: 787,968 params
2025-11-04 15:00:15 - processors.model_manager - INFO -   combined_modulator: 

Predicting: |          | 0/? [00:00<?, ?it/s]

2025-11-04 15:00:21 - filelock - DEBUG - Attempting to acquire lock 128109017523904 on /work/_artifacts/model/common/reference_genomes/cres_seqs_manifest.parquet.lock
2025-11-04 15:00:21 - filelock - DEBUG - Attempting to acquire lock 128109017522272 on /work/_artifacts/model/common/reference_genomes/cres_seqs_manifest.parquet.lock
2025-11-04 15:00:21 - filelock - DEBUG - Attempting to acquire lock 128109017520256 on /work/_artifacts/model/common/reference_genomes/cres_seqs_manifest.parquet.lock
2025-11-04 15:00:21 - filelock - DEBUG - Lock 128109017523904 not acquired on /work/_artifacts/model/common/reference_genomes/cres_seqs_manifest.parquet.lock, waiting 0.05 seconds ...
2025-11-04 15:00:21 - filelock - DEBUG - Lock 128109017526880 acquired on /work/_artifacts/model/common/reference_genomes/cres_seqs_manifest.parquet.lock
2025-11-04 15:00:21 - filelock - DEBUG - Lock 128109017520256 not acquired on /work/_artifacts/model/common/reference_genomes/cres_seqs_manifest.parquet.lock, wa

Formatting VariantFormer scores and computing eQTL statistics...
Computing eQTL statistics...


In [11]:
final_results.head()

,variant_id,genes,tissues,ref,alt,chr,pos,VF-agg-log2fc-weighted,VF-AFR-2-exp-log2fc,VF-AMR-2-exp-log2fc,VF-EAS-2-exp-log2fc,VF-EUR-2-exp-log2fc,VF-REF_HG38-2-exp-log2fc,VF-SAS-2-exp-log2fc
0,chr10_100236603_C_T,ENSG00000095485.16,adipose - subcutaneous,C,T,chr10,100236603,0.009466,0.000000,0.027943,0.000000,0.000000,0.007029,0.007029
1,chr10_100236603_C_T,ENSG00000095485.16,whole blood,C,T,chr10,100236603,0.017387,0.060486,0.006830,0.020407,0.020407,0.000000,0.020407
2,chr10_100238657_T_C,ENSG00000095485.16,adipose - subcutaneous,T,C,chr10,100238657,0.009475,0.000000,0.027943,0.000000,0.000000,0.007029,0.007029
3,chr10_100238657_T_C,ENSG00000095485.16,whole blood,T,C,chr10,100238657,0.017376,0.060486,0.006830,0.020407,0.020407,0.006830,0.020407
4,chr10_100239989_G_A,ENSG00000095485.16,adipose - subcutaneous,G,A,chr10,100239989,0.009435,0.000000,0.027943,0.000000,0.000000,0.000000,0.007029


## Launch predictions using Alphagenome

### Load data

In [12]:
variants = pd.read_parquet(os.path.join(REPO_PATH, '_artifacts/b23c9b69.pq'))

In [13]:
# tiisue map to align tissue names between VF and Eqtl catalog data
alphagenome_tissue_map = {'skin' : 'Skin_Sun_Exposed_Lower_leg',
                 'blood' : 'Whole_Blood',
                 'adipose' : 'Adipose_Subcutaneous',
                 'brain - frontal cortex (ba9)': 'Brain_Frontal_Cortex_BA9',
                 'brain - putamen (basal ganglia)': 'Brain_Putamen_basal_ganglia',
                 'brain - substantia nigra' : 'Brain_Substantia_nigra'
                 }
variants['tissue'] = variants['tissue'].map(alphagenome_tissue_map)

In [14]:
first10_variants = variants.head(10)
first10_variants['CHROM'] = first10_variants['variant_id'].apply(lambda x: x.split('_')[0])
first10_variants['POS'] = first10_variants['variant_id'].apply(lambda x: int(x.split('_')[1]))
first10_variants['REF'] = first10_variants['variant_id'].apply(lambda x: x.split('_')[2])
first10_variants['ALT'] = first10_variants['variant_id'].apply(lambda x: x.split('_')[3])
first10_variants['gene_id'] = first10_variants['gene_id'].apply(lambda x: x.split('.')[0])

In [15]:
# Install alphagenome
!uv pip install alphagenome
from io import StringIO
from alphagenome import colab_utils
from alphagenome.data import genome
from alphagenome.models import dna_client, variant_scorers
import numpy as np

Using Python 3.12.10 environment at: /work/.venv
Resolved 41 packages in 669ms
   Building intervaltree==3.1.0
      Built intervaltree==3.1.0
Prepared 11 packages in 458ms
Installed 11 packages in 19ms
 + absl-py==2.3.1
 + alphagenome==0.4.0
 + grpcio==1.76.0
 + immutabledict==4.2.2
 + intervaltree==3.1.0
 + jaxtyping==0.3.3
 + ml-dtypes==0.5.3
 + sortedcontainers==2.4.0
 + typeguard==4.4.4
 + wadler-lindig==0.1.7
 + zstandard==0.25.0


2025-11-04 15:01:08 - h5py._conv - DEBUG - Creating converter from 7 to 5
2025-11-04 15:01:08 - h5py._conv - DEBUG - Creating converter from 5 to 7
2025-11-04 15:01:08 - h5py._conv - DEBUG - Creating converter from 7 to 5
2025-11-04 15:01:08 - h5py._conv - DEBUG - Creating converter from 5 to 7
2025-11-04 15:01:08 - numcodecs - DEBUG - Registering codec 'zlib'
2025-11-04 15:01:08 - numcodecs - DEBUG - Registering codec 'gzip'
2025-11-04 15:01:08 - numcodecs - DEBUG - Registering codec 'bz2'
2025-11-04 15:01:08 - numcodecs - DEBUG - Registering codec 'lzma'
2025-11-04 15:01:08 - numcodecs - DEBUG - Registering codec 'blosc'
2025-11-04 15:01:08 - numcodecs - DEBUG - Registering codec 'zstd'
2025-11-04 15:01:08 - numcodecs - DEBUG - Registering codec 'lz4'
2025-11-04 15:01:08 - numcodecs - DEBUG - Registering codec 'astype'
2025-11-04 15:01:08 - numcodecs - DEBUG - Registering codec 'delta'
2025-11-04 15:01:08 - numcodecs - DEBUG - Registering codec 'quantize'
2025-11-04 15:01:08 - numcod

In [ ]:
# Load the model.
API_KEY = "your_api_key_here"
dna_model = dna_client.create(API_KEY)

In [ ]:
# Alphagenome specifications
organism = 'human'
# @markdown Specify length of sequence around variants to predict:
sequence_length = '1MB'
sequence_length = dna_client.SUPPORTED_SEQUENCE_LENGTHS[
    f'SEQUENCE_LENGTH_{sequence_length}'
]
score_rna_seq = True
download_predictions = False
# Parse organism specification.
organism_map = {
    'human': dna_client.Organism.HOMO_SAPIENS,
    'mouse': dna_client.Organism.MUS_MUSCULUS,
}
organism = organism_map[organism]
# Parse scorer specification.
scorer_selections = {
    'rna_seq': score_rna_seq,
}
all_scorers = variant_scorers.RECOMMENDED_VARIANT_SCORERS
selected_scorers = [
    all_scorers[key]
    for key in all_scorers
    if scorer_selections.get(key.lower(), False)
]


In [ ]:
results = []
for i, vcf_row in first10_variants.iterrows():
  variant = genome.Variant(
      chromosome=str(vcf_row.CHROM),
      position=int(vcf_row.POS),
      reference_bases=vcf_row.REF,
      alternate_bases=vcf_row.ALT,
      name=vcf_row.variant_id,
  )
  interval = variant.reference_interval.resize(sequence_length)

  variant_scores = dna_model.score_variant(
      interval=interval,
      variant=variant,
      variant_scorers=selected_scorers,
      organism=organism,
  )
  df_scores = variant_scorers.tidy_scores([variant_scores])
  tissue_key = vcf_row.tissue
  df = df_scores[(df_scores['gene_id'] == vcf_row.gene_id) & (df_scores['gtex_tissue'] == tissue_key)]['raw_score']
  vcf_row['alphagenome_score_new'] = df.values[0].mean() if not df.empty else np.nan
  results.append(vcf_row)

In [ ]:
df = pd.DataFrame(results)

**The alphagenome score might deviate a little because we believe the underlying model is evolving** 

## Visualize all the precalculated scores

In [16]:
variants = pd.read_parquet(os.path.join(REPO_PATH, '_artifacts/b23c9b69.pq'))

In [17]:
variants.head()

,variant_id,gene_id,tissue,slope,pvalue,ref,alt,maf,VF-agg-log2fc-weighted_ag,VF-AFR-2-log2fc_ag,...,VF-SAS-2-log2fc_ag,VF-agg-log2fc-weighted_pcg,VF-AFR-2-log2fc_pcg,VF-AMR-2-log2fc_pcg,VF-EAS-2-log2fc_pcg,VF-EUR-2-log2fc_pcg,VF-SAS-2-log2fc_pcg,variant_id_gene,Borzoi-log2FC,alphagenome_score
0,chr10_100236603_C_T,ENSG00000095485.16,adipose,0.659835,2.606480e-38,C,T,0.429134,0.009466,0.000000,...,0.007029,0.012622,0.000000,0.041915,0.000000,0.00000,0.000000,chr10_100236603_C_T_ENSG00000095485.16,-0.000599,0.000815
1,chr10_100236603_C_T,ENSG00000095485.16,blood,0.476792,8.194630e-18,C,T,0.410256,0.017387,0.060486,...,0.020407,0.007350,0.033332,0.006720,0.020079,0.00000,0.020079,chr10_100236603_C_T_ENSG00000095485.16,-0.000461,0.000996
2,chr10_100238657_T_C,ENSG00000095485.16,adipose,0.659753,2.629150e-38,T,C,0.429134,0.009475,0.000000,...,0.007029,0.013702,0.000000,0.041915,0.000000,0.00000,0.007058,chr10_100238657_T_C_ENSG00000095485.16,0.000460,-0.000954
3,chr10_100238657_T_C,ENSG00000095485.16,blood,0.476752,8.216580e-18,T,C,0.410256,0.017376,0.060486,...,0.020407,0.011797,0.033332,0.006720,0.026719,0.00672,0.026719,chr10_100238657_T_C_ENSG00000095485.16,0.000305,-0.000968
4,chr10_100239989_G_A,ENSG00000095485.16,adipose,0.659729,2.636980e-38,G,A,0.429134,0.009435,0.000000,...,0.007029,0.012604,0.000000,0.041915,0.000000,0.00000,0.000000,chr10_100239989_G_A_ENSG00000095485.16,0.001238,0.000725


### VF results

In [18]:
# Tissue specific spoearman correlation plots
vf_spearman = variants.groupby('tissue').apply(
    lambda x: x['slope'].corr(x['VF-agg-log2fc-weighted_ag'], method='spearman')
).reset_index()
vf_spearman.columns = ['tissue', 'spearman_correlation']
vf_spearman


,tissue,spearman_correlation
0,adipose,0.540437
1,blood,0.453173
2,brain - frontal cortex (ba9),0.625611
3,brain - putamen (basal ganglia),0.607318
4,brain - substantia nigra,0.587621
5,skin,0.605895


### Alphagenome results

In [19]:
# Tissue specific spoearman correlation plots
alphagenome_spearman = variants.groupby('tissue').apply(
    lambda x: x['slope'].corr(x['alphagenome_score'], method='spearman')
).reset_index()
alphagenome_spearman.columns = ['tissue', 'spearman_correlation']
alphagenome_spearman

,tissue,spearman_correlation
0,adipose,0.039658
1,blood,0.035846
2,brain - frontal cortex (ba9),0.044307
3,brain - putamen (basal ganglia),0.016789
4,brain - substantia nigra,0.018051
5,skin,0.019730
